In [1]:
import pandas as pd
file_path = '../data/digit-recognizer/train.csv' 
digits_images = pd.read_csv(file_path)



In [2]:
import numpy as np
from sklearn.cluster import KMeans

def process_image_data_to_df(df):
    # Lista para armazenar resultados intermediários
    processed_data = []

    for idx, row in df.iterrows():
        # 1. Transformar linha em uma imagem bidimensional (ignorando o rótulo da classe)
        label = row['label']  # Rótulo da classe
        pixels = row[1:].values.reshape(28, 28)  # Pixels (28x28)
        
        # 1.1 Filtrar os pixels com valor > 0 e obter coordenadas
        coords = np.column_stack(np.where(pixels > 0))  # Coordenadas [x1, x2]
        
        media_img = coords.mean(axis=0)
                
        kmeans = KMeans(n_clusters=10, init='k-means++', random_state=42) # Colocar random state como hyper parâmetro
        kmeans.fit(coords)
        centroids = kmeans.cluster_centers_
        
        # 3. Normalizar os centroides com base na média
        normalized_centroids = centroids - media_img
        
        # Adicionar os resultados processados como uma linha
        processed_data.append({
            'label': label,
            **{f'c{i}_x': normalized_centroids[i][0] for i in range(10)},
            **{f'c{i}_y': normalized_centroids[i][1] for i in range(10)},
        })
    
    return pd.DataFrame(processed_data)
processed_df = process_image_data_to_df(digits_images.iloc[:1000, :])



In [11]:
from scipy.optimize import linear_sum_assignment

def reorder_centroids(reference, target):
    """
    Reordena os centróides da imagem `target` para alinhar com `reference`.

    Args:
        reference (ndarray): Centróides de referência (10x2).
        target (ndarray): Centróides da imagem alvo (10x2).

    Returns:
        reordered_target (ndarray): Centróides reordenados.
    """
    # Calcular a matriz de custo (distância Euclidiana entre centróides)
    cost_matrix = np.linalg.norm(reference[:, np.newaxis] - target[np.newaxis, :], axis=2)
    
    # Encontrar o melhor assignment com o algoritmo de Hungarian
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    
    # Reordenar os centróides da `target` de acordo com o assignment
    reordered_target = target[col_ind]
    
    return reordered_target

def reorder_all_centroids(df, reference_by_class):
    """
    Reordena os centróides de todas as imagens no DataFrame para alinhar com a referência por classe.

    Args:
        df (DataFrame): DataFrame com centróides para cada imagem.
        reference_by_class (dict): Um dicionário contendo os centróides de referência por classe.

    Returns:
        df_reordered (DataFrame): DataFrame com centróides reordenados.
    """
    reordered_data = []
    
    for _, row in df.iterrows():
        label = row['label']
        centroids = row[1:].to_numpy().reshape(2,10).T
        
        # Reordenar os centróides para a imagem atual
        reordered_centroids = reorder_centroids(reference_by_class, centroids)
        
        # Adicionar ao novo DataFrame
        reordered_row = {'label': label}
        for i, centroid in enumerate(reordered_centroids):
            reordered_row[f'c{i}_x'] = centroid[0]
            reordered_row[f'c{i}_y'] = centroid[1]
        reordered_data.append(reordered_row)
    
    return pd.DataFrame(reordered_data)

principal_centroids = []
class_references = []
for num_label in range(10):
    filtered_df = processed_df[processed_df['label'] == num_label]
    ref_row_np = filtered_df.iloc[0,1:].to_numpy().reshape(2,10).T
    class_references.append(ref_row_np)
    centroids_reordered = reorder_all_centroids(filtered_df, ref_row_np)
    column_means = centroids_reordered.mean()
    centroids_means = [] 
    for i in range(10):
        x = column_means[f'c{i}_x']
        y = column_means[f'c{i}_y']
        centroids_means.append([x, y])
    principal_centroids.append(np.array(centroids_means))

In [20]:
guesses = [] 
for _, row in digits_images.iloc[:1000, :].iterrows():
    label = row['label']  # Rótulo da classe
    pixels = row[1:].values.reshape(28, 28)  # Pixels (28x28)
    
    coords = np.column_stack(np.where(pixels > 0))  # Coordenadas [x1, x2]
        
    media_img = coords.mean(axis=0)
            
    kmeans = KMeans(n_clusters=10, init='k-means++', random_state=42)
    kmeans.fit(coords)
    centroids = kmeans.cluster_centers_
    
    normalized_centroids = centroids - media_img
        
    row_dists = [] 
    for i in range(10):
        reordered_centroids = reorder_centroids(principal_centroids[i], centroids)
        row_dists.append(np.linalg.norm(np.array(reordered_centroids) - principal_centroids[i]))
    guesses.append([np.argmin(np.array(row_dists)), label])  

guesses = np.array(guesses)
count_equal = np.sum(guesses[:, 0] == guesses[:, 1])
print(count_equal)

254
